## SRJ

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                print('Exc. dealing right now is', ash_excitation[i])
                print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        print('Moving towards parameters')
        #params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
        #print('The length of parameters is', params)
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs_state, H, qubits


In [2]:
energies, params, ash_excitation, gs_state,H, qubits = adaptvqe(adapt_it = 8, e_th=1e-12)
print('Energies:', energies)


Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668924
The adapt iteration now is 0
The current excitation operator is a⁺(0) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(4) is 9.258183414698196e-08
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 9.258183412269583e-08
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(4) is 0.0
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 1.1698969991369434e-07
The current excitation operator is a⁺(3) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(5) is 0.0
The current excitation operator is a⁺(3) a(7)
Print, if this is activated - HF state


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(3) a(7) is 1.1698969991369434e-07
The current excitation operator is a⁺(0) a⁺(1) a(4) a(5)
Print, if this is activated - HF state


...

The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.5196913803431857e-17
The current excitation operator is a⁺(2) a⁺(3) a(5) a(6)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.2188137010702093e-17
The current excitation operator is a⁺(2) a⁺(3) a(6) a(7)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.03354401712245733
The highest operator value is 0.03354401712245733 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
The current status of ash_excitation is [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5], [2, 6], [3, 7], [2, 3, 6, 7]]
Moving towards parameters
Parameters before optimization are [-0.82176297 -1.33664719 -0.9776199  -1.20616273  0.31947365  1.56457721
 -1.28274059  0.        ]
Current parameters: [-0.82174366 -1.33665704 -0.9776177  -1.20616505  0.31947255  1.56457888
 -1.28274572  0.07819452]
Current cost: -1.8663329835988478

Current parameters: [-0.83161335 -1.34430364 -0.97065533 -1.21191462  0.31899391  1.56468342
 -1.28620006  0.08110543]
Current cost: -1.866366083097988

Current parameters: [-0.83533183 -1.34656917 -0.96749543 -1.21316277  0.31905162  1.56797514
 -1.28943828  0.0821946 ]
Current cost: -1.8663726621450465

Current parameters: [-0.8367544  -1.34724965 

In [3]:
paramss = np.array([0.1] * (2 * qubits), requires_grad=True)
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def k2(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.expval(H)

print('Params before optimizeation', paramss)

print(qml.draw(k2, max_length=200)(paramss))


optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

for n in range(450):
    paramss, energy = optimizer.step_and_cost(k2, paramss)
    print('The 1st set params after calculation :', paramss)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
print(qml.draw(k2, max_length=200)(paramss))

Params before optimizeation

 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0: ─╭|Ψ⟩──RY(0.10)─╭●──RY(0.10)─────────────────────────────────────────────────────────────┤ ╭<𝓗>
1: ─├|Ψ⟩──RY(0.10)─╰X─╭●─────────RY(0.10)───────────────────────────────────────────────────┤ ├<𝓗>
2: ─├|Ψ⟩──RY(0.10)────╰X────────╭●─────────RY(0.10)─────────────────────────────────────────┤ ├<𝓗>
3: ─├|Ψ⟩──RY(0.10)──────────────╰X────────╭●─────────RY(0.10)───────────────────────────────┤ ├<𝓗>
4: ─├|Ψ⟩──RY(0.10)────────────────────────╰X────────╭●─────────RY(0.10)─────────────────────┤ ├<𝓗>
5: ─├|Ψ⟩──RY(0.10)──────────────────────────────────╰X────────╭●─────────RY(0.10)───────────┤ ├<𝓗>
6: ─├|Ψ⟩──RY(0.10)────────────────────────────────────────────╰X────────╭●─────────RY(0.10)─┤ ├<𝓗>
7: ─╰|Ψ⟩──RY(0.10)──────────────────────────────────────────────────────╰X─────────RY(0.10)─┤ ╰<𝓗>
The 1st set params after calculation : [0.10352985 0.1036012  0.10640919 0.10562353 0.09616059 0.10650525
 0.10550564 0.10695863 0.10022244 0

In [4]:
paramss

tensor([-0.05841735, -0.08268292, -0.02417075,  0.50905265, -1.58483882,
         1.62816597,  1.27353716,  1.34101924,  1.95499636, -0.03880251,
         0.12400676, -1.28055779, -1.53902113,  0.41216993,  1.25485811,
        -1.5859973 ], requires_grad=True)

In [5]:
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def ostatehea(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.state()
fstate = ostatehea(paramss)
print(qml.draw(ostatehea, max_length=200)(paramss))

0: ─╭|Ψ⟩──RY(-0.06)─╭●──RY(1.95)─────────────────────────────────────────────────────────────────┤  State
1: ─├|Ψ⟩──RY(-0.08)─╰X─╭●─────────RY(-0.04)──────────────────────────────────────────────────────┤  State
2: ─├|Ψ⟩──RY(-0.02)────╰X────────╭●──────────RY(0.12)────────────────────────────────────────────┤  State
3: ─├|Ψ⟩──RY(0.51)───────────────╰X─────────╭●─────────RY(-1.28)─────────────────────────────────┤  State
4: ─├|Ψ⟩──RY(-1.58)─────────────────────────╰X────────╭●──────────RY(-1.54)──────────────────────┤  State
5: ─├|Ψ⟩──RY(1.63)────────────────────────────────────╰X─────────╭●──────────RY(0.41)────────────┤  State
6: ─├|Ψ⟩──RY(1.27)───────────────────────────────────────────────╰X─────────╭●─────────RY(1.25)──┤  State
7: ─╰|Ψ⟩──RY(1.34)──────────────────────────────────────────────────────────╰X─────────RY(-1.59)─┤  State


In [6]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe_updated(fstate, adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for GS state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - GS state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, fstate, H):
        #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        qml.StatePrep(fstate, wires=range(qubits))
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(fstate, ash_excitation, params):
        #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        qml.StatePrep(fstate, wires=range(qubits))
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, fstate, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [fstate]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else fstate  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, fstate): # If the current state is the HF state
                print('Print, if this is activated - fstate')
                #print('The state now is', k)
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(fstate, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(fstate,ash_excitation,params))
        gs1_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs1_state, H, qubits









In [7]:
energies, params, ash_excitation, gs1_state,H, qubits = adaptvqe_updated(fstate, adapt_it = 10, e_th=1e-12)
print('Energies:', energies)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668922
The adapt iteration now is 0
The current excitation operator is a⁺(0) a(4)
Print, if this is activated - fstate
The expectation value of a⁺(0) a(4) is 0.008336412243142789
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - fstate


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(0) a(6) is 0.037770898373310755
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - fstate
The expectation value of a⁺(1) a(5) is 0.044528200890278875
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - fstate
The expectation value of a⁺(1) a(7) is 0.0031645406329839864
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - fstate


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a(4) is 0.009678574755070023
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - fstate
The expectation value of a⁺(2) a(6) is 0.020831365307549643
The current excitation operator is a⁺(3) a(5)


...

The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 0.0034823923221116455
The current excitation operator is a⁺(2) a⁺(3) a(5) a(6)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 0.0037658448969856055
The current excitation operator is a⁺(2) a⁺(3) a(6) a(7)


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.03924948090496743
The highest operator value is 0.048296778207411385 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]
The current status of ash_excitation is [[1, 2, 4, 7], [0, 3, 5, 6], [0, 1, 4, 5], [0, 4], [0, 3, 5, 6], [0, 3, 5, 6], [0, 3, 5, 6], [0, 3, 5, 6], [0, 3, 5, 6], [0, 3, 5, 6]]
Parameters before optimization are [ 1.32729837e+00 -8.52171647e-01  2.63873749e+00  4.06009861e-01
  2.33615840e-01  1.66930941e-07  1.30872789e-07 -5.25014557e-08
  8.02818737e-08  0.00000000e+00]
Current parameters: [ 1.32729837e+00 -8.52172207e-01  2.63873749e+00  4.06009861e-01
  2.33615840e-01  1.66930941e-07  1.30885072e-07  3.68727944e-07
  8.04152616e-08  0.00000000e+00]
Current cost: -1.5243549531082434

Optimization terminated successfully.
         Current function value: -1.524355
         Iterations: 1
         Function evaluations: 290
Final updated parameters: [ 1.32729837e+00 -8.52172207e-01  2.638737

In [9]:
ash_excitation

[[1, 2, 4, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [0, 4],
 [0, 3, 5, 6],
 [0, 3, 5, 6],
 [0, 3, 5, 6],
 [0, 3, 5, 6],
 [0, 3, 5, 6],
 [0, 3, 5, 6]]